<a href="https://colab.research.google.com/github/rsr2425/word-count-investigation/blob/main/notebooks/1_Naive Approach GPT 3_5 Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

test test test

> NOTE: You only need a CPU instance to run this notebook since it relies on OpenAI API calls to perform any LLM related work.

# Setup

In [6]:
# download library code
!git clone https://github.com/rsr2425/word-count-investigation.git

Cloning into 'word-count-investigation'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 32 (delta 4), reused 28 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (32/32), 42.44 KiB | 1.52 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [7]:
%cd word-count-investigation

/content/word-count-investigation/word-count-investigation


In [8]:
CURRENT_MODEL_ID = "gpt-3.5-turbo"
LLM_JUDGE_MODEL_ID = "gpt-4o"
PROJECT_NAME = "word-count-investigation"

In [9]:
!pip install datasets langchain_openai rouge-score evaluate wandb #deepeval

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=9dacd5ec5e35ac24d78faa603fdb0be45f9c6ffb682f069aba11c01a22024607
  

In [11]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

## Dataset

In [12]:
from datasets import load_dataset

dataset = load_dataset("ccdv/cnn_dailymail", '3.0.0', split="test[:1000]")
dataset = dataset.rename_column('article', 'text')
dataset = dataset.rename_column('highlights', 'summary')
dataset = dataset.remove_columns(['id'])

README.md:   0%|          | 0.00/13.9k [00:00<?, ?B/s]

cnn_dailymail.py:   0%|          | 0.00/9.27k [00:00<?, ?B/s]

cnn_stories.tgz:   0%|          | 0.00/159M [00:00<?, ?B/s]

dailymail_stories.tgz:   0%|          | 0.00/376M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [13]:
dataset

Dataset({
    features: ['text', 'summary'],
    num_rows: 1000
})

## Metrics

# Experiments

In [18]:
from experiments import run_experiment

ImportError: cannot import name 'run_experiment' from 'experiments' (/content/word-count-investigation/experiments/__init__.py)

In [17]:
# Parameters across runs
SUBSET_SIZE = 100 # if set to None, entire dataset will be processed
TEMPERATURE = 0.7
NUMBER_OF_QUESTIONS = 10
LOG_TO_WANDB = False

metrics = [
    Metric.ROUGE,
    # Metric.FACTUAL_CONSISTENCY,
]

ModuleNotFoundError: No module named 'experiments.experiments'

## Run: Baseline

In [ ]:
# Run Parameters
WORD_COUNT_TARGET = None
RUN_PREFIX=f"baseline_"

results = run_experiment(
    CURRENT_MODEL_ID,
    TEMPERATURE,
    dataset,
    NUMBER_OF_QUESTIONS,
    metrics,
    word_count_target=WORD_COUNT_TARGET,
    subset_size=SUBSET_SIZE,
    log_to_wandb=LOG_TO_WANDB,
)
df = results.to_pandas()
df.select_dtypes(include='number').mean()

Processing Dataset!
Now summarizing data...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Now calculating Metric.ROUGE...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Done!


,0
text_word_count,693.520000
summary_word_count,35.550000
ai_summary_word_count,115.320000
rouge1,0.241629
rouge2,0.084713
rougeL,0.163744
rougeLsum,0.196493


## Run: Generate with Target Word Count(25)

In [ ]:
# Run Parameters
WORD_COUNT_TARGET = 25
RUN_PREFIX=f"word_cnt_target_{WORD_COUNT_TARGET}_"

results = run_experiment(
    CURRENT_MODEL_ID,
    TEMPERATURE,
    dataset,
    NUMBER_OF_QUESTIONS,
    metrics,
    word_count_target=WORD_COUNT_TARGET,
    subset_size=SUBSET_SIZE,
    log_to_wandb=LOG_TO_WANDB,
)
df = results.to_pandas()
df.select_dtypes(include='number').mean()

Processing Dataset!
Now summarizing data...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Now calculating Metric.ROUGE...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Done!


,0
text_word_count,693.520000
summary_word_count,35.550000
ai_summary_word_count,71.230000
rouge1,0.273371
rouge2,0.088202
rougeL,0.183531
rougeLsum,0.218301


## Run: Generate with Target Word Count(50)

In [ ]:
WORD_COUNT_TARGET = 50
RUN_PREFIX=f"word_cnt_target_{WORD_COUNT_TARGET}_"

results = run_experiment(
    CURRENT_MODEL_ID,
    TEMPERATURE,
    dataset,
    NUMBER_OF_QUESTIONS,
    metrics,
    word_count_target=WORD_COUNT_TARGET,
    subset_size=SUBSET_SIZE,
    log_to_wandb=LOG_TO_WANDB,
)
df = results.to_pandas()
df.select_dtypes(include='number').mean()

Processing Dataset!
Now summarizing data...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Now calculating Metric.ROUGE...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Done!


,0
text_word_count,693.520000
summary_word_count,35.550000
ai_summary_word_count,71.330000
rouge1,0.276339
rouge2,0.083146
rougeL,0.181936
rougeLsum,0.214358


## Run: Generate with Target Word Count(75)

In [ ]:
WORD_COUNT_TARGET = 75
RUN_PREFIX=f"word_cnt_target_{WORD_COUNT_TARGET}_"

results = run_experiment(
    CURRENT_MODEL_ID,
    TEMPERATURE,
    dataset,
    NUMBER_OF_QUESTIONS,
    metrics,
    word_count_target=WORD_COUNT_TARGET,
    subset_size=SUBSET_SIZE,
    log_to_wandb=LOG_TO_WANDB,
)
df = results.to_pandas()
df.select_dtypes(include='number').mean()

Processing Dataset!
Now summarizing data...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Now calculating Metric.ROUGE...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Done!


,0
text_word_count,693.520000
summary_word_count,35.550000
ai_summary_word_count,70.300000
rouge1,0.281052
rouge2,0.084944
rougeL,0.182206
rougeLsum,0.217115


## Run: Generate with Target Word Count(100)

In [ ]:
WORD_COUNT_TARGET = 100
RUN_PREFIX=f"word_cnt_target_{WORD_COUNT_TARGET}_"

results = run_experiment(
    CURRENT_MODEL_ID,
    TEMPERATURE,
    dataset,
    NUMBER_OF_QUESTIONS,
    metrics,
    word_count_target=WORD_COUNT_TARGET,
    subset_size=SUBSET_SIZE,
    log_to_wandb=LOG_TO_WANDB,
)
df = results.to_pandas()
df.select_dtypes(include='number').mean()

Processing Dataset!
Now summarizing data...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Now calculating Metric.ROUGE...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Done!


,0
text_word_count,693.520000
summary_word_count,35.550000
ai_summary_word_count,69.220000
rouge1,0.281300
rouge2,0.085715
rougeL,0.182644
rougeLsum,0.218219


## Run: Generate with Target Word Count(150)

In [ ]:
WORD_COUNT_TARGET = 150
RUN_PREFIX=f"word_cnt_target_{WORD_COUNT_TARGET}_"

results = run_experiment(
    CURRENT_MODEL_ID,
    TEMPERATURE,
    dataset,
    NUMBER_OF_QUESTIONS,
    metrics,
    word_count_target=WORD_COUNT_TARGET,
    subset_size=SUBSET_SIZE,
    log_to_wandb=LOG_TO_WANDB,
)
df = results.to_pandas()
df.select_dtypes(include='number').mean()

Processing Dataset!
Now summarizing data...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Now calculating Metric.ROUGE...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Done!


,0
text_word_count,693.520000
summary_word_count,35.550000
ai_summary_word_count,69.130000
rouge1,0.281399
rouge2,0.089064
rougeL,0.187815
rougeLsum,0.227144


## Run: Generate with subjective language(concise)

In [ ]:
# Run Parameters
WORD_COUNT_TARGET = None
RUN_PREFIX=f"subjective_concise_"

results = run_experiment(
    CURRENT_MODEL_ID,
    TEMPERATURE,
    dataset,
    NUMBER_OF_QUESTIONS,
    metrics,
    word_count_target=WORD_COUNT_TARGET,
    subset_size=SUBSET_SIZE,
    otherinstructions="Make the summary concise.",
    log_to_wandb=LOG_TO_WANDB,
)
df = results.to_pandas()
df.select_dtypes(include='number').mean()

Processing Dataset!
Now summarizing data...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Now calculating Metric.ROUGE...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Done!


,0
text_word_count,693.520000
summary_word_count,35.550000
ai_summary_word_count,118.850000
rouge1,0.241762
rouge2,0.087490
rougeL,0.163201
rougeLsum,0.197492


## Run: Generate with subjective language(2 sent.)

In [ ]:
# Run Parameters
WORD_COUNT_TARGET = None
RUN_PREFIX=f"subjective_2_sent_"

results = run_experiment(
    CURRENT_MODEL_ID,
    TEMPERATURE,
    dataset,
    NUMBER_OF_QUESTIONS,
    metrics,
    word_count_target=WORD_COUNT_TARGET,
    subset_size=SUBSET_SIZE,
    otherinstructions="Make the summary two sentences or less.",
    log_to_wandb=LOG_TO_WANDB,
)
df = results.to_pandas()
df.select_dtypes(include='number').mean()

Processing Dataset!
Now summarizing data...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Now calculating Metric.ROUGE...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Done!


,0
text_word_count,693.520000
summary_word_count,35.550000
ai_summary_word_count,117.870000
rouge1,0.242911
rouge2,0.086394
rougeL,0.161455
rougeLsum,0.192476


## Run: Generate with subjective language(1 sent.)

In [ ]:
# Run Parameters
WORD_COUNT_TARGET = None
RUN_PREFIX=f"subjective_1_sent_"

results = run_experiment(
    CURRENT_MODEL_ID,
    TEMPERATURE,
    dataset,
    NUMBER_OF_QUESTIONS,
    metrics,
    word_count_target=WORD_COUNT_TARGET,
    subset_size=SUBSET_SIZE,
    otherinstructions="Make the summary one sentence long.",
    log_to_wandb=LOG_TO_WANDB,
)
df = results.to_pandas()
df.select_dtypes(include='number').mean()

Processing Dataset!
Now summarizing data...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Now calculating Metric.ROUGE...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Done!


,0
text_word_count,693.520000
summary_word_count,35.550000
ai_summary_word_count,118.650000
rouge1,0.243341
rouge2,0.085500
rougeL,0.164883
rougeLsum,0.196868


## Run: Generate with subjective language(brief)

In [ ]:
# Run Parameters
WORD_COUNT_TARGET = None
RUN_PREFIX=f"subjective_brief_"

results = run_experiment(
    CURRENT_MODEL_ID,
    TEMPERATURE,
    dataset,
    NUMBER_OF_QUESTIONS,
    metrics,
    word_count_target=WORD_COUNT_TARGET,
    subset_size=SUBSET_SIZE,
    otherinstructions="Make the summary brief.",
    log_to_wandb=LOG_TO_WANDB,
)
df = results.to_pandas()
df.select_dtypes(include='number').mean()

Processing Dataset!
Now summarizing data...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Now calculating Metric.ROUGE...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Done!


...

 ··········


wandb: ERROR API key must be 40 characters long, yours was 9


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


,0
text_word_count,693.520000
summary_word_count,35.550000
ai_summary_word_count,120.160000
rouge1,0.237304
rouge2,0.084362
rougeL,0.158038
rougeLsum,0.193628


## Run: Generate with Alt Target Word Cnt Language

In [ ]:
# Run Parameters
WORD_COUNT_TARGET = None
RUN_PREFIX=f"word_cnt_target_alt_{WORD_COUNT_TARGET}_"

results = run_experiment(
    CURRENT_MODEL_ID,
    TEMPERATURE,
    dataset,
    NUMBER_OF_QUESTIONS,
    metrics,
    word_count_target=WORD_COUNT_TARGET,
    subset_size=SUBSET_SIZE,
    otherinstructions="Use exact 30 words in your summary.",
    log_to_wandb=LOG_TO_WANDB,
)
df = results.to_pandas()
df.select_dtypes(include='number').mean()

Processing Dataset!
Now summarizing data...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Now calculating Metric.ROUGE...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Done!


,0
text_word_count,693.520000
summary_word_count,35.550000
ai_summary_word_count,123.250000
rouge1,0.236568
rouge2,0.086404
rougeL,0.156627
rougeLsum,0.189498


## Run: Generate with Alt Target Token Cnt Language

In [ ]:
# Run Parameters
WORD_COUNT_TARGET = None
RUN_PREFIX=f"token_cnt_target_alt_{WORD_COUNT_TARGET}_"

results = run_experiment(
    CURRENT_MODEL_ID,
    TEMPERATURE,
    dataset,
    NUMBER_OF_QUESTIONS,
    metrics,
    word_count_target=WORD_COUNT_TARGET,
    subset_size=SUBSET_SIZE,
    otherinstructions="Use exact 70 token in your summary.",
    log_to_wandb=LOG_TO_WANDB,
)
df = results.to_pandas()
df.select_dtypes(include='number').mean()

Processing Dataset!
Now summarizing data...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Now calculating Metric.ROUGE...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Done!


,0
text_word_count,693.520000
summary_word_count,35.550000
ai_summary_word_count,119.210000
rouge1,0.240759
rouge2,0.084506
rougeL,0.163756
rougeLsum,0.195359


## Run: Generate with Token Target

## Run: Generate with Token Target (Revise)

In [ ]:
# # Run Parameters
# WORD_COUNT_TARGET = None
# RUN_PREFIX=f"baseline_"

# results = run_experiment(
#     "gpt-4o",
#     TEMPERATURE,
#     dataset,
#     NUMBER_OF_QUESTIONS,
#     metrics,
#     word_count_target=WORD_COUNT_TARGET,
#     subset_size=SUBSET_SIZE,
#     log_to_wandb=LOG_TO_WANDB,
# )
# df = results.to_pandas()
# df.select_dtypes(include='number').mean()

Processing Dataset!
Now summarizing data...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Now calculating Metric.ROUGE...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Done!


,0
text_word_count,693.520000
summary_word_count,35.550000
ai_summary_word_count,125.240000
rouge1,0.231237
rouge2,0.078238
rougeL,0.149871
rougeLsum,0.181594


In [ ]:
# # Run Parameters
# WORD_COUNT_TARGET = 25
# RUN_PREFIX=f"word_cnt_target_{WORD_COUNT_TARGET}_"
# metrics = [
#     Metric.ROUGE,
#     Metric.FACTUAL_CONSISTENCY,
# ]

# results = run_experiment(
#     "gpt-4o",
#     TEMPERATURE,
#     dataset,
#     NUMBER_OF_QUESTIONS,
#     metrics,
#     word_count_target=WORD_COUNT_TARGET,
#     subset_size=SUBSET_SIZE,
#     log_to_wandb=LOG_TO_WANDB,
# )
# df = results.to_pandas()
# df.select_dtypes(include='number').mean()

Processing Dataset!
Now summarizing data...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Now calculating Metric.ROUGE...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Now calculating Metric.FACTUAL_CONSISTENCY...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Done!


,0
text_word_count,693.520000
summary_word_count,35.550000
ai_summary_word_count,83.360000
rouge1,0.269485
rouge2,0.085634
rougeL,0.179384
rougeLsum,0.212510
human_factual_consistency,0.000000
ai_factual_consistency,0.000000


In [ ]:
# # Run Parameters
# WORD_COUNT_TARGET = 25
# RUN_PREFIX=f"word_cnt_target_{WORD_COUNT_TARGET}_"

# results = run_experiment(
#     "gpt-4",
#     TEMPERATURE,
#     dataset,
#     NUMBER_OF_QUESTIONS,
#     metrics,
#     word_count_target=WORD_COUNT_TARGET,
#     subset_size=SUBSET_SIZE,
#     log_to_wandb=LOG_TO_WANDB,
# )
# df = results.to_pandas()
# df.select_dtypes(include='number').mean()

Processing Dataset!
Now summarizing data...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Now calculating Metric.ROUGE...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Done!


,0
text_word_count,693.520000
summary_word_count,35.550000
ai_summary_word_count,80.540000
rouge1,0.294006
rouge2,0.105077
rougeL,0.195565
rougeLsum,0.233589


In [ ]:
results[0]

{'text': '(CNN)James Best, best known for his portrayal of bumbling sheriff Rosco P. Coltrane on TV\'s "The Dukes of Hazzard," died Monday after a brief illness. He was 88. Best died in hospice in Hickory, North Carolina, of complications from pneumonia, said Steve Latshaw, a longtime friend and Hollywood colleague. Although he\'d been a busy actor for decades in theater and in Hollywood, Best didn\'t become famous until 1979, when "The Dukes of Hazzard\'s" cornpone charms began beaming into millions of American homes almost every Friday night. For seven seasons, Best\'s Rosco P. Coltrane chased the moonshine-running Duke boys back and forth across the back roads of fictitious Hazzard County, Georgia, although his "hot pursuit" usually ended with him crashing his patrol car. Although Rosco was slow-witted and corrupt, Best gave him a childlike enthusiasm that got laughs and made him endearing. His character became known for his distinctive "kew-kew-kew" chuckle and for goofy catchphras